In [1]:
# encoding:utf-8
# 这里用urllib在有些网页上会有'utf-8' codec can't decode byte 0xb0 in position 38281: invalid start byte的问题，主要是因为
# 返回的网页以gzip压缩过的数据，所以要进行解码，所以要用from io import BytesIO，import gzip这两个包，但是requests库不会有这样的
# 问题，因为requests库会自动对返回的网页内容进行解码
import urllib
import re
from io import BytesIO
import gzip
import requests

emaillist = []
QQlist = []
urllist = []
def gettiebalistnumbers(name):
    url = "http://tieba.baidu.com/f?"
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36"}
    word = {"kw": name}
    word = urllib.parse.urlencode(word)
    url = url + word
    request = urllib.request.Request(url, headers=headers)
    request.add_header("Connection", "keep-alive")
    response = urllib.request.urlopen(request)
    data = response.read().decode("utf-8")
    #     帖子数量
    restr = '<span class=\"card_infoNum\">([\S\s]*?)</span>'
    regex = re.compile(restr, re.IGNORECASE)
    mylist = regex.findall(data)
    tie_numbers = mylist[0].replace(",", "")
    tie_numbers = eval(tie_numbers) #转化为数字
    #     关注人数
    restr2 = '<span class=\"card_menNum\">([\S\s]*?)</span>'
    regex2 = re.compile(restr2, re.IGNORECASE)
    mylist2 = regex2.findall(data)
    men_numbers = mylist2[0].replace(",", "")
    men_numbers = eval(men_numbers) #转化为数字
    return tie_numbers, men_numbers

def gettiebalist(name):
    numberstuple = gettiebalistnumbers(name)
    tienumbers = numberstuple[1] #帖子的数量
    tiebalist = [] #每页的url列表
    if tienumbers%50 == 0: #生成页面列表
        for i in range(tienumbers//50):
            tiebalist.append("http://tieba.baidu.com/f?kw="+name+"&ie=utf-8&pn="+str(i*50))
    else:
        for i in range(tienumbers//50+1):
            tiebalist.append("http://tieba.baidu.com/f?kw="+name+"&ie=utf-8&pn="+str(i*50))
    return tiebalist

def geturllistfrompage(url):
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36"}
    request = urllib.request.Request(url, headers=headers)
    response = urllib.request.urlopen(request)
    data = response.read().decode("utf-8")
    restr = '<ul id=\"thread_list\" class=\"threadlist_bright j_threadlist_bright\">([\S\s]*?)<div class=\"thread_list_bottom clearfix\">'
    regex = re.compile(restr, re.IGNORECASE)
    mylist = regex.findall(data)
    tablestr = mylist[0]
    restr = 'href="/p/(\d+)"' #取每一个帖子的url值
    regex = re.compile(restr, re.IGNORECASE)
    urltitlelist = regex.findall(tablestr)
    for title in urltitlelist:
        urllist.append("http://tieba.baidu.com/p/"+title)

# f1 = open('emaillist.txt','w',encoding='utf-8')    #文件操作符 文件句柄 文件操作对象
def getemaillistfrompage(url):
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36"}
    request = urllib.request.Request(url, headers=headers)
    response = urllib.request.urlopen(request)
    response2 = requests.get(url, headers=headers)
    response2.encoding = "utf-8"
#     content = response.read()              # content是压缩过的数据
#     buff = BytesIO(content)            # 把content转为文件对象
#     f = gzip.GzipFile(fileobj=buff)
#     data = f.read().decode('utf-8')
    data = response2.text
    restr = r"\w{0,19}@[0-9a-zA-Z]{1,13}\.[com,cn,net]{1,3}"
    regex = re.compile(restr, re.IGNORECASE)
    qqnumber = regex.findall(data)
    emaillist.append(qqnumber)
#     f1.write(qqnumber)

# f2 = open('qqlist.txt','w',encoding='utf-8')    #文件操作符 文件句柄 文件操作对象
def getQQlistfrompage(url):
    header = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36"}
    request = urllib.request.Request(url, headers=header)
    response = urllib.request.urlopen(request)
    response2 = requests.get(url, headers=header)
    response2.encoding = "utf-8"
#     content = response.read()              # content是压缩过的数据
#     buff = BytesIO(content)            # 把content转为文件对象
#     f = gzip.GzipFile(fileobj=buff)
    while True:
        line = response2.text
        if not line:
            break
        if line.find("QQ") != -1 or line.find("Qq") != -1 or line.find("qq") != -1:
            restr = "[1-9][0-9]{4,14}" #或者"[1-9]\\d{4,14}"
            regex = re.compile(restr, re.IGNORECASE)
            title = regex.findall(line)
            for each in title:
#                 QQlist.extend(regex.findall(line))
                QQlist.append(each)
#                 f2.write(each)

print("贴数  %d  关注人数  %d" %(gettiebalistnumbers("python3")))
tiezi_list = gettiebalist("python3")
for line in tiezi_list:
    print(line)
    geturllistfrompage(line)
for url in urllist: 
#     geturllistfrompage("http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=0")
    getemaillistfrompage(url)
    getQQlistfrompage(url)
# for i in emaillist:
#     print(i)
# print("-------------------------------------------------------------------")
# for j in QQlist:
#     print(j)

贴数  27737  关注人数  11070
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=0
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=50
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=100
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=150
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=200
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=250
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=300
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=350
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=400
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=450
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=500
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=550
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=600
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=650
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=700
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=750
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=800
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=850
http://tieba.baidu.com/f?kw=python3&ie=utf-8

http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=7750
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=7800
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=7850
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=7900
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=7950
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=8000
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=8050
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=8100
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=8150
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=8200
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=8250
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=8300
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=8350
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=8400
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=8450
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=8500
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=8550
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=8600
http://tieba.baidu.com/f?kw=python3&ie=utf-8&p

MemoryError: 

In [ ]:
print(len(emaillist))
print(len(QQlist))

In [8]:
# coding:utf-8
import urllib
import re
from http import cookiejar

url_baidu_index = "http://www.baidu.com"
url_baidu_token = "https://passport.baidu.com/v2/api/?getapi&tpl=pp&apiver=v3&class=login"
url_baidu_login = "https://passport.baidu.com/v2/api/?login"
user = "18981651893"
password = "8611662"
filepath = "cookie.txt"
cookiejar = cookiejar.CookieJar() #cookie管理器
opener = urllib.request.build_opener(urllib.request.HTTPCookieProcessor(cookiejar))
urllib.request.install_opener(opener) #全局生效，urllib打开
reqReturn = urllib.request.urlopen(url_baidu_index) #打开主页
# 抓取token
tokenReturn = urllib.request.urlopen(url_baidu_token)
matchval = re.search(u'"token" : "(?P<tokenVal>.*?)"',tokenReturn.read().decode('utf-8'))
tokenVal = matchval.group("tokenVal")
print(tokenVal)
postdata = {
    'username':user,
    'password':password,
    'u':"https://passport.baidu.com/",
    'tpl':'pp',
    'token':tokenVal,
    'staticpage':'https://passport.baidu.com/static/passpc-account/html/v3Jump.html',
    'isPhone':'false',
    'charset':"utf-8",
    'callback':"parent.bd__pcbs__ra48vi"
}
postdata = urllib.parse.urlencode(postdata) #编码转换
loginRequest = urllib.request.Request(url_baidu_login, postdata) #模拟登录
#增加请求头，模拟真实浏览器
loginRequest.add_header('Accept', 'application/json, text/javascript, */*; q=0.01')
loginRequest.add_header('Accept-Encoding', 'gzip, deflate')
loginRequest.add_header('Accept-Language', 'zh-CN,zh;q=0.9')
loginRequest.add_header('User-Agent', 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36')
loginRequest.add_header('Content-Type', 'application/x-www-form-urlencoded')

#发起登录请求
selfindex = "http://tieba.baidu.com/home/main?id=ac07e5ae81e69a97e79a84e5aeb6533e?t=1473561554&fr=userbar&red_tag=b0141861968"
content = urllib.request.urlopen(selfindex).read().decode('utf-8')
print(content.find("宁暗的家"))
print(content)
# cookie = cookiejar.LWPCookieJar(filepath) #设定保存路径
# header = urllib.request.HTTPCookieProcessor(cookie) #设置cookie头，与网站有关
# opener = urllib.request.build_opener(header)
# response = opener.open("http://www.baidu.com")
# cookie.save(ignore_expires=True, ignore_discard=True) #忽略

7c4bdeca92e0507f33584a38a3a53c8e
307
<!DOCTYPE html><!--STATUS OK--><html><head><meta charset="UTF-8"><meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1"><meta name="baidu-site-verification" content="jpBCrwX689" /><link rel="search" type="application/opensearchdescription+xml" href="/tb/cms/content-search.xml" title="百度贴吧" /><title>宁暗的家的贴吧</title><script type="text/javascript">void function(t,e,n,a,o,i,r){t.alogObjectName=o,t[o]=t[o]||function(){(t[o].q=t[o].q||[]).push(arguments)},t[o].l=t[o].l||+new Date,a="https:"===t.location.protocol?"https://fex.bdstatic.com"+a:"http://fex.bdstatic.com"+a;var c=!0;if(t.alogObjectConfig&&t.alogObjectConfig.sample){var s=Math.random();t.alogObjectConfig.rand=s,s>t.alogObjectConfig.sample&&(c=!1)}c&&(i=e.createElement(n),i.async=!0,i.src=a+"?v="+~(new Date/864e5)+~(new Date/864e5),r=e.getElementsByTagName(n)[0],r.parentNode.insertBefore(i,r))}(window,document,"script","/hunter/alog/alog.min.js","alog"),void function(){function t(){}window.

In [ ]:
# encoding:utf-8
# 这里用urllib在有些网页上会有'utf-8' codec can't decode byte 0xb0 in position 38281: invalid start byte的问题，主要是因为
# 返回的网页以gzip压缩过的数据，所以要进行解码，所以要用from io import BytesIO，import gzip这两个包，但是requests库不会有这样的
# 问题，因为requests库会自动对返回的网页内容进行解码
import urllib
import re
from io import BytesIO
import gzip
import requests

emaillist = []
QQlist = []
urllist = []
def gettiebalistnumbers(name):
    url = "http://tieba.baidu.com/f?"
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36"}
    word = {"kw": name}
    word = urllib.parse.urlencode(word)
    url = url + word
    request = urllib.request.Request(url, headers=headers)
    request.add_header("Connection", "keep-alive")
    response = urllib.request.urlopen(request)
    data = response.read().decode("utf-8")
    #     帖子数量
    restr = '<span class=\"card_infoNum\">([\S\s]*?)</span>'
    regex = re.compile(restr, re.IGNORECASE)
    mylist = regex.findall(data)
    tie_numbers = mylist[0].replace(",", "")
    tie_numbers = eval(tie_numbers) #转化为数字
    #     关注人数
    restr2 = '<span class=\"card_menNum\">([\S\s]*?)</span>'
    regex2 = re.compile(restr2, re.IGNORECASE)
    mylist2 = regex2.findall(data)
    men_numbers = mylist2[0].replace(",", "")
    men_numbers = eval(men_numbers) #转化为数字
    return tie_numbers, men_numbers

def gettiebalist(name):
    numberstuple = gettiebalistnumbers(name)
    tienumbers = numberstuple[1] #帖子的数量
    tiebalist = [] #每页的url列表
    if tienumbers%50 == 0: #生成页面列表
        for i in range(tienumbers//50):
            tiebalist.append("http://tieba.baidu.com/f?kw="+name+"&ie=utf-8&pn="+str(i*50))
    else:
        for i in range(tienumbers//50+1):
            tiebalist.append("http://tieba.baidu.com/f?kw="+name+"&ie=utf-8&pn="+str(i*50))
    return tiebalist

def geturllistfrompage(url):
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36"}
    request = urllib.request.Request(url, headers=headers)
    response = urllib.request.urlopen(request)
    data = response.read().decode("utf-8")
    restr = '<ul id=\"thread_list\" class=\"threadlist_bright j_threadlist_bright\">([\S\s]*?)<div class=\"thread_list_bottom clearfix\">'
    regex = re.compile(restr, re.IGNORECASE)
    mylist = regex.findall(data)
    tablestr = mylist[0]
    restr = 'href="/p/(\d+)"' #取每一个帖子的url值
    regex = re.compile(restr, re.IGNORECASE)
    urltitlelist = regex.findall(tablestr)
    for title in urltitlelist:
        urllist.append("http://tieba.baidu.com/p/"+title)

f1 = open('emaillist.txt','w',encoding='utf-8')    #文件操作符 文件句柄 文件操作对象
def getemaillistfrompage(url):
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36"}
    request = urllib.request.Request(url, headers=headers)
    response = urllib.request.urlopen(request)
    response2 = requests.get(url, headers=headers)
    response2.encoding = "utf-8"
#     content = response.read()              # content是压缩过的数据
#     buff = BytesIO(content)            # 把content转为文件对象
#     f = gzip.GzipFile(fileobj=buff)
#     data = f.read().decode('utf-8')
    data = response2.text
    restr = "\w{0,19}@[0-9a-zA-Z]{1,13}\.[com,cn,net]{1,3}"
    regex = re.compile(restr, re.IGNORECASE)
    emailnumber = regex.findall(data)
#     emaillist.append(qqnumber)
    for Emailnumber in emailnumber:
#         f1.write(Emailnumber)
        print(Emailnumber)
    del emailnumber

f2 = open('qqlist.txt','w',encoding='utf-8')    #文件操作符 文件句柄 文件操作对象
def getQQlistfrompage(url):
    header = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36"}
    request = urllib.request.Request(url, headers=header)
    response = urllib.request.urlopen(request)
    response2 = requests.get(url, headers=header)
    response2.encoding = "utf-8"
#     content = response.read()              # content是压缩过的数据
#     buff = BytesIO(content)            # 把content转为文件对象
#     f = gzip.GzipFile(fileobj=buff)
    while True:
        line = response2.text
        if not line:
            break
        if line.find("QQ") != -1 or line.find("Qq") != -1 or line.find("qq") != -1:
            restr = "[1-9][0-9]{4,14}" #或者"[1-9]\\d{4,14}"
            regex = re.compile(restr, re.IGNORECASE)
            title = regex.findall(line)
            for each in title:
#                 QQlist.extend(regex.findall(line))
#                 QQlist.append(each)
                f2.write(each)
#                 print(each)
            del title

print("贴数  %d  关注人数  %d" %(gettiebalistnumbers("python3")))
tiezi_list = gettiebalist("python3")
for line in tiezi_list:
    print(line)
    geturllistfrompage(line)
for url in urllist: 
    getemaillistfrompage(url)
    getQQlistfrompage(url)
f1.close()
f2.close()
# for i in emaillist:
#     print(i)
# print("-------------------------------------------------------------------")
# for j in QQlist:
#     print(j)

贴数  27791  关注人数  11085
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=0
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=50
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=100
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=150
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=200
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=250
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=300
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=350
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=400
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=450
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=500
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=550
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=600
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=650
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=700
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=750
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=800
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=850
http://tieba.baidu.com/f?kw=python3&ie=utf-8

http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=7750
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=7800
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=7850
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=7900
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=7950
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=8000
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=8050
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=8100
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=8150
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=8200
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=8250
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=8300
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=8350
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=8400
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=8450
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=8500
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=8550
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=8600
http://tieba.baidu.com/f?kw=python3&ie=utf-8&p

In [1]:
# encoding:utf-8
# 这里用urllib在有些网页上会有'utf-8' codec can't decode byte 0xb0 in position 38281: invalid start byte的问题，主要是因为
# 返回的网页以gzip压缩过的数据，所以要进行解码，所以要用from io import BytesIO，import gzip这两个包，但是requests库不会有这样的
# 问题，因为requests库会自动对返回的网页内容进行解码
import urllib
import re
from io import BytesIO
import gzip
import requests
import http.client
http.client.HTTPConnection._http_vsn = 10
http.client.HTTPConnection._http_vsn_str = 'HTTP/1.0'

emaillist = []
QQlist = []
urllist = []
def gettiebalistnumbers(name):
    url = "http://tieba.baidu.com/f?"
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36"}
    word = {"kw": name}
    word = urllib.parse.urlencode(word)
    url = url + word
    request = urllib.request.Request(url, headers=headers)
    request.add_header("Connection", "keep-alive")
    response = urllib.request.urlopen(request)
    data = response.read().decode("utf-8")
    #     帖子数量
    restr = '<span class=\"card_infoNum\">([\S\s]*?)</span>'
    regex = re.compile(restr, re.IGNORECASE)
    mylist = regex.findall(data)
    tie_numbers = mylist[0].replace(",", "")
    tie_numbers = eval(tie_numbers) #转化为数字
    #     关注人数
    restr2 = '<span class=\"card_menNum\">([\S\s]*?)</span>'
    regex2 = re.compile(restr2, re.IGNORECASE)
    mylist2 = regex2.findall(data)
    men_numbers = mylist2[0].replace(",", "")
    men_numbers = eval(men_numbers) #转化为数字
    return tie_numbers, men_numbers

def gettiebalist(name):
    numberstuple = gettiebalistnumbers(name)
    tienumbers = numberstuple[1] #帖子的数量
    tiebalist = [] #每页的url列表
    if tienumbers%50 == 0: #生成页面列表
        for i in range(tienumbers//50):
            tiebalist.append("http://tieba.baidu.com/f?kw="+name+"&ie=utf-8&pn="+str(i*50))
    else:
        for i in range(tienumbers//50+1):
            tiebalist.append("http://tieba.baidu.com/f?kw="+name+"&ie=utf-8&pn="+str(i*50))
    return tiebalist

def geturllistfrompage(url):
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36"}
    request = urllib.request.Request(url, headers=headers)
    response = urllib.request.urlopen(request)
#     data = response.read().decode("utf-8")
    try:
        data = response.read().decode("utf-8")
    except Exception as e:
        page = e.partial
        data = page.decode('utf-8')
    restr = '<ul id=\"thread_list\" class=\"threadlist_bright j_threadlist_bright\">([\S\s]*?)<div class=\"thread_list_bottom clearfix\">'
    regex = re.compile(restr, re.IGNORECASE)
    mylist = regex.findall(data)
    tablestr = mylist[0]
    restr = 'href="/p/(\d+)"' #取每一个帖子的url值
    regex = re.compile(restr, re.IGNORECASE)
    urltitlelist = regex.findall(tablestr)
    for title in urltitlelist:
        urllist.append("http://tieba.baidu.com/p/"+title)

f1 = open('emaillist.txt','w',encoding='utf-8')    #文件操作符 文件句柄 文件操作对象
def getemaillistfrompage(url):
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36"}
    request = urllib.request.Request(url, headers=headers)
    response = urllib.request.urlopen(request)
    response2 = requests.get(url, headers=headers)
    response2.encoding = "utf-8"
    data = response2.text
    restr = "\w{0,19}@[0-9a-zA-Z]{1,13}\.[com,cn,net]{1,3}"
    regex = re.compile(restr, re.IGNORECASE)
    emailnumber = regex.findall(data)
#     emaillist.append(qqnumber)
    for Emailnumber in emailnumber:
        f1.write(Emailnumber+"\n")
#         print(Emailnumber)
    del emailnumber

f2 = open('qqlist.txt','w',encoding='utf-8')    #文件操作符 文件句柄 文件操作对象
def getQQlistfrompage(url):
    header = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36"}
    request = urllib.request.Request(url, headers=header)
    response = urllib.request.urlopen(request)
    response2 = requests.get(url, headers=header)
    response2.encoding = "utf-8"
    line = response2.text
    restr = "qq:[1-9][0-9]{4,14}" #或者"[1-9]\\d{4,14}"
    regex = re.compile(restr, re.IGNORECASE)
    title = regex.findall(line)
    for each in title:
#         QQlist.extend(regex.findall(line))
#         QQlist.append(each)
        f2.write(each+"\n")
#         print(each)
    del title

print("贴数  %d  关注人数  %d" %(gettiebalistnumbers("python3")))
tiezi_list = gettiebalist("python3")
for line in tiezi_list:
    print(line)
    geturllistfrompage(line)
for url in urllist:
    print(url)
    getemaillistfrompage(url)
    getQQlistfrompage(url)
f1.close()
f2.close()
# for i in emaillist:
#     print(i)
# print("-------------------------------------------------------------------")
# for j in QQlist:
#     print(j)

贴数  27828  关注人数  11098
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=0
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=50
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=100
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=150
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=200
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=250
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=300
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=350
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=400
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=450
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=500
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=550
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=600
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=650
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=700
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=750
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=800
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=850
http://tieba.baidu.com/f?kw=python3&ie=utf-8

http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=7750
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=7800
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=7850
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=7900
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=7950
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=8000
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=8050
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=8100
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=8150
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=8200
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=8250
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=8300
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=8350
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=8400
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=8450
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=8500
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=8550
http://tieba.baidu.com/f?kw=python3&ie=utf-8&pn=8600
http://tieba.baidu.com/f?kw=python3&ie=utf-8&p

http://tieba.baidu.com/p/5983605606
http://tieba.baidu.com/p/5983052960
http://tieba.baidu.com/p/5983394813
http://tieba.baidu.com/p/5971952970
http://tieba.baidu.com/p/5982930147
http://tieba.baidu.com/p/5982928005
http://tieba.baidu.com/p/5687088515
http://tieba.baidu.com/p/5982836044
http://tieba.baidu.com/p/5978991607
http://tieba.baidu.com/p/5969152808
http://tieba.baidu.com/p/5881030579
http://tieba.baidu.com/p/5981939765
http://tieba.baidu.com/p/5981795512
http://tieba.baidu.com/p/5981321864
http://tieba.baidu.com/p/5977941848
http://tieba.baidu.com/p/5981093016
http://tieba.baidu.com/p/5738508225
http://tieba.baidu.com/p/5979072657
http://tieba.baidu.com/p/5892773778
http://tieba.baidu.com/p/5979558776
http://tieba.baidu.com/p/5976711572
http://tieba.baidu.com/p/5979127548
http://tieba.baidu.com/p/5979126996
http://tieba.baidu.com/p/5977304818
http://tieba.baidu.com/p/5978322038
http://tieba.baidu.com/p/5973633311
http://tieba.baidu.com/p/5977966306
http://tieba.baidu.com/p/597

http://tieba.baidu.com/p/5930482677
http://tieba.baidu.com/p/5933874401
http://tieba.baidu.com/p/5585665083
http://tieba.baidu.com/p/5929399414
http://tieba.baidu.com/p/5508571325
http://tieba.baidu.com/p/5484483916
http://tieba.baidu.com/p/5931071275
http://tieba.baidu.com/p/5932387412
http://tieba.baidu.com/p/5931686683
http://tieba.baidu.com/p/5931286346
http://tieba.baidu.com/p/4803771344
http://tieba.baidu.com/p/5931071492
http://tieba.baidu.com/p/5930287083
http://tieba.baidu.com/p/5930849568
http://tieba.baidu.com/p/5911255630
http://tieba.baidu.com/p/5929484657
http://tieba.baidu.com/p/5929738157
http://tieba.baidu.com/p/5674920277
http://tieba.baidu.com/p/5929575922
http://tieba.baidu.com/p/5929446409
http://tieba.baidu.com/p/5927371491
http://tieba.baidu.com/p/5925572812
http://tieba.baidu.com/p/5926879002
http://tieba.baidu.com/p/5926152508
http://tieba.baidu.com/p/5152148724
http://tieba.baidu.com/p/5928358322
http://tieba.baidu.com/p/5927011548
http://tieba.baidu.com/p/564

http://tieba.baidu.com/p/5834865981
http://tieba.baidu.com/p/5805865784
http://tieba.baidu.com/p/5820354309
http://tieba.baidu.com/p/5829259028
http://tieba.baidu.com/p/5834034949
http://tieba.baidu.com/p/5843585052
http://tieba.baidu.com/p/5850502928
http://tieba.baidu.com/p/5849484677
http://tieba.baidu.com/p/5841212364
http://tieba.baidu.com/p/5849531698
http://tieba.baidu.com/p/5848116380
http://tieba.baidu.com/p/5848525131
http://tieba.baidu.com/p/5848303055
http://tieba.baidu.com/p/5286732748
http://tieba.baidu.com/p/5845888762
http://tieba.baidu.com/p/5819657782
http://tieba.baidu.com/p/5844400553
http://tieba.baidu.com/p/5844286029
http://tieba.baidu.com/p/5839161162
http://tieba.baidu.com/p/5824702065
http://tieba.baidu.com/p/5836774566
http://tieba.baidu.com/p/5820009485
http://tieba.baidu.com/p/5842902606
http://tieba.baidu.com/p/5842873290
http://tieba.baidu.com/p/4799667706
http://tieba.baidu.com/p/5842302089
http://tieba.baidu.com/p/5839627926
http://tieba.baidu.com/p/582

...

http://tieba.baidu.com/p/1728915149
http://tieba.baidu.com/p/1658602209
http://tieba.baidu.com/p/1714747632
http://tieba.baidu.com/p/1703242424
http://tieba.baidu.com/p/1675551578
http://tieba.baidu.com/p/1717848699
http://tieba.baidu.com/p/1658555159
http://tieba.baidu.com/p/1659351836
http://tieba.baidu.com/p/4488986251
http://tieba.baidu.com/p/1707191782
http://tieba.baidu.com/p/4491413453
http://tieba.baidu.com/p/4490008262
http://tieba.baidu.com/p/4464740423
http://tieba.baidu.com/p/4479789912
http://tieba.baidu.com/p/4461765957
http://tieba.baidu.com/p/4484018062
http://tieba.baidu.com/p/4480907789
http://tieba.baidu.com/p/4481270634
http://tieba.baidu.com/p/4479488702
http://tieba.baidu.com/p/4477579034
http://tieba.baidu.com/p/4074810106
http://tieba.baidu.com/p/4473608290
http://tieba.baidu.com/p/4474365420
http://tieba.baidu.com/p/4471061717
http://tieba.baidu.com/p/4472341125
http://tieba.baidu.com/p/4472849916
http://tieba.baidu.com/p/1658885738
http://tieba.baidu.com/p/447

http://tieba.baidu.com/p/1701127472
http://tieba.baidu.com/p/3555491777
http://tieba.baidu.com/p/3619682779
http://tieba.baidu.com/p/1658865857
http://tieba.baidu.com/p/4348680122
http://tieba.baidu.com/p/3060712573
http://tieba.baidu.com/p/4116101945
http://tieba.baidu.com/p/3416813963
http://tieba.baidu.com/p/2855437751
http://tieba.baidu.com/p/2641699656
http://tieba.baidu.com/p/2022891990
http://tieba.baidu.com/p/1716347532
http://tieba.baidu.com/p/2377380822
http://tieba.baidu.com/p/1660066829
http://tieba.baidu.com/p/1658533623
http://tieba.baidu.com/p/1862609724
http://tieba.baidu.com/p/1730634151
http://tieba.baidu.com/p/1728915149
http://tieba.baidu.com/p/1658602209
http://tieba.baidu.com/p/1714747632
http://tieba.baidu.com/p/1703242424
http://tieba.baidu.com/p/1675551578
http://tieba.baidu.com/p/1717848699
http://tieba.baidu.com/p/1658555159
http://tieba.baidu.com/p/1659351836
http://tieba.baidu.com/p/4488986251
http://tieba.baidu.com/p/1707191782
http://tieba.baidu.com/p/449

http://tieba.baidu.com/p/4484018062
http://tieba.baidu.com/p/4480907789
http://tieba.baidu.com/p/4481270634
http://tieba.baidu.com/p/4479488702
http://tieba.baidu.com/p/4477579034
http://tieba.baidu.com/p/4074810106
http://tieba.baidu.com/p/4473608290
http://tieba.baidu.com/p/4474365420
http://tieba.baidu.com/p/4471061717
http://tieba.baidu.com/p/4472341125
http://tieba.baidu.com/p/4472849916
http://tieba.baidu.com/p/1658885738
http://tieba.baidu.com/p/4470010001
http://tieba.baidu.com/p/4469628608
http://tieba.baidu.com/p/4466731886
http://tieba.baidu.com/p/4466096095
http://tieba.baidu.com/p/4465268646
http://tieba.baidu.com/p/1701127472
http://tieba.baidu.com/p/3555491777
http://tieba.baidu.com/p/3619682779
http://tieba.baidu.com/p/1658865857
http://tieba.baidu.com/p/4348680122
http://tieba.baidu.com/p/3060712573
http://tieba.baidu.com/p/4116101945
http://tieba.baidu.com/p/3416813963
http://tieba.baidu.com/p/2855437751
http://tieba.baidu.com/p/2641699656
http://tieba.baidu.com/p/202

http://tieba.baidu.com/p/1862609724
http://tieba.baidu.com/p/1730634151
http://tieba.baidu.com/p/1728915149
http://tieba.baidu.com/p/1658602209
http://tieba.baidu.com/p/1714747632
http://tieba.baidu.com/p/1703242424
http://tieba.baidu.com/p/1675551578
http://tieba.baidu.com/p/1717848699
http://tieba.baidu.com/p/1658555159
http://tieba.baidu.com/p/1659351836
http://tieba.baidu.com/p/4488986251
http://tieba.baidu.com/p/1707191782
http://tieba.baidu.com/p/4491413453
http://tieba.baidu.com/p/4490008262
http://tieba.baidu.com/p/4464740423
http://tieba.baidu.com/p/4479789912
http://tieba.baidu.com/p/4461765957
http://tieba.baidu.com/p/4484018062
http://tieba.baidu.com/p/4480907789
http://tieba.baidu.com/p/4481270634
http://tieba.baidu.com/p/4479488702
http://tieba.baidu.com/p/4477579034
http://tieba.baidu.com/p/4074810106
http://tieba.baidu.com/p/4473608290
http://tieba.baidu.com/p/4474365420
http://tieba.baidu.com/p/4471061717
http://tieba.baidu.com/p/4472341125
http://tieba.baidu.com/p/447

http://tieba.baidu.com/p/4466096095
http://tieba.baidu.com/p/4465268646
http://tieba.baidu.com/p/1701127472
http://tieba.baidu.com/p/3555491777
http://tieba.baidu.com/p/3619682779
http://tieba.baidu.com/p/1658865857
http://tieba.baidu.com/p/4348680122
http://tieba.baidu.com/p/3060712573
http://tieba.baidu.com/p/4116101945
http://tieba.baidu.com/p/3416813963
http://tieba.baidu.com/p/2855437751
http://tieba.baidu.com/p/2641699656
http://tieba.baidu.com/p/2022891990
http://tieba.baidu.com/p/1716347532
http://tieba.baidu.com/p/2377380822
http://tieba.baidu.com/p/1660066829
http://tieba.baidu.com/p/1658533623
http://tieba.baidu.com/p/1862609724
http://tieba.baidu.com/p/1730634151
http://tieba.baidu.com/p/1728915149
http://tieba.baidu.com/p/1658602209
http://tieba.baidu.com/p/1714747632
http://tieba.baidu.com/p/1703242424
http://tieba.baidu.com/p/1675551578
http://tieba.baidu.com/p/1717848699
http://tieba.baidu.com/p/1658555159
http://tieba.baidu.com/p/1659351836
http://tieba.baidu.com/p/448